In [ ]:
class DictsList:
    def __init__(self, dictslist, listname):
    #inialist list object with file contests
        
    def show_dicts():
    
    def delete_dict():
        
    def new_dict():
        
    def append_dictslist():
        
    def update_dictslist():